# Price Tracker
---


Tracks products on German "Kleinanzeigen" website.

This script is for training only.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Step 1: Ask for product-name and keywords

**Example Product**: Nikon D 7500

**Example Keywords**: Neu Objektiv Kit OVP

In [135]:
product_input = str(input('Which product are you looking for?'))
i = 0
product = ''

for words in product_input.split(' '):
    if i == (len(product_input.split(' '))-1):
        product = product + words
    else: 
        product = product + words + '-'
    i += 1

keys_input = str(input('Which keywords would you like to filter for? (no comma, max. 4)'))
i = 0
keys = []

for words in keys_input.split(' '):
    keys += [words]

## Step 2: Create Product list for specific product

In [ ]:
url = f'https://www.kleinanzeigen.de/s-{product}/k0'

# count number of subpages

p = requests.get(url)
website_data = BeautifulSoup(p.text, 'html.parser')
number_of_pages = len(website_data.find_all('a', {'class': 'pagination-page'}))

# get productlist

def get_productlist(url):
    
    # check each page
    for i in range(1,number_of_pages): 
        if i > 1: 
            url = f'https://www.kleinanzeigen.de/s-seite:{i}/{product}/k0'
        
        # get html content
        r = requests.get(url)
        html_content = BeautifulSoup(r.text, 'html.parser')
        
        # get productlist
        if i == 1:
            productslist = []
        results = html_content.find_all('div', {'class': 'aditem-main'})
        for item in results:
            if item.find('a', {'class': 'ellipsis'}) is not None:
                products = {
                    'title': item.find('a', {'class': 'ellipsis'}).text.replace(',','.'),
                    'price': int(item.find('p', {'class': 'aditem-main--middle--price-shipping--price'}).text.replace(' ','').replace('\n','').replace('.','').split("€")[0].replace('VB','0').replace('"','0')),
                    'link': item.find('a', {'class': 'ellipsis'})['href']
                }
            productslist.append(products)
    return productslist

def create_dataframe(productslist):
    df_products = pd.DataFrame(productslist)
    df_products = df_products.sort_values(by=['price'])
    df_products.to_csv('products_all.csv', index=False)
    print('You are looking for: ' + product_input)
    print(len(df_products), 'products saved to CSV')
    return df_products

df_products = create_dataframe(get_productlist(url))

You are looking for: Nikon D 7500
50 products saved to CSV


## Step 3: Filter for specific content

In [180]:
# optional: reduce list products with keywords

print('Your keywords are:', keys)
i = 0

def find_keywords(df_products):
    df_products_key = pd.DataFrame()
    for entry in df_products['title']:
        if keys[0] in entry or keys[1] in entry or keys[2] in entry or keys[3] in entry:
            filtered_entry = df_products[df_products['title'] == entry]
            df_products_key = pd.concat([df_products_key, filtered_entry], ignore_index=True)
    df_products_key.to_csv('products_key.csv', index=False)
    print(len(df_products_key), 'filtered products saved to CSV')

find_keywords(df_products)

Your keywords are: ['neu', 'Objektiv', 'OVP', 'Kit']
14 filtered products saved to CSV


## Step 4: Get product description of each product

In [197]:
# add column in dataframe
df_products_detailed = df_products
df_products_detailed["description"] = ' '
i = 0

# check each page
for subpage in df_products['link']:
    if type(subpage) == str:
        url = 'https://www.kleinanzeigen.de' + subpage
        description = ''
        q = requests.get(url)
        html_content = BeautifulSoup(q.text, 'html.parser')
        if html_content.find('p', {'id': 'viewad-description-text'}) is not None:
            description = html_content.find('p', {'id': 'viewad-description-text'}).text.replace('\n',' ').replace('  ','')
        df_products_detailed.loc[i,'description'] = description
        i += 1

df_products_detailed.to_csv('products_detailed.csv', index=False)